# SQLite

In [5]:
import sqlite3
import json
from typing import Dict, Any

class Database:
    def __init__(self, db_path: str):
        """
        Initialize SQLite database connection.
        :param db_path: Path to the SQLite .db file
        """
        try:
            self.connection = sqlite3.connect(db_path)
            # Enable accessing rows as dicts
            self.connection.row_factory = sqlite3.Row
            self.cursor = self.connection.cursor()
            print("🔌 Database connection successful")
        except sqlite3.Error as e:
            print(f"❌ Database connection failed: {e}")
            raise

    def _create_table_if_not_exists(self):
        """
        Ensure the user_profiles table exists with the correct schema.
        """
        try:
            self.cursor.execute(
                """
                CREATE TABLE IF NOT EXISTS user_profiles (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    age INTEGER,
                    gender TEXT,
                    marital_status TEXT,
                    education TEXT,
                    job TEXT,
                    location TEXT,
                    first_interest TEXT,
                    first_interest_percentage INTEGER,
                    second_interest TEXT,
                    second_interest_percentage INTEGER,
                    third_interest TEXT,
                    third_interest_percentage INTEGER,
                    personality_summary TEXT,
                    key_activities TEXT,
                    total_posts INTEGER,
                    top_habits TEXT,
                    top_hobby TEXT,
                    travel_indicators TEXT,
                    life_indicators TEXT,
                    spending_indicators TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
                """
            )
            self.connection.commit()
            print("✅ Table verified/created successfully")
        except sqlite3.Error as e:
            print(f"❌ Table creation failed: {e}")
            raise

    def insert_user_profile(self, profile_data: Dict[str, Any]) -> bool:
        """
        Insert a user profile into the database with transaction handling.
        :param profile_data: Dictionary containing user profile fields and interest lists
        :return: True on success, False otherwise
        """
        try:
            self._create_table_if_not_exists()

            interests = profile_data.get("top_interests", [{} for _ in range(3)])

            sql = (
                """
                INSERT INTO user_profiles (
                    first_name, last_name, age, gender, marital_status, education, job, location,
                    first_interest, first_interest_percentage,
                    second_interest, second_interest_percentage,
                    third_interest, third_interest_percentage,
                    personality_summary, key_activities, total_posts, top_habits,
                    top_hobby, travel_indicators, life_indicators, spending_indicators
                ) VALUES (
                    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
                )
                """
            )

            values = (
                profile_data.get("first_name", ""),
                profile_data.get("last_name", ""),
                profile_data.get("age"),
                profile_data.get("gender", ""),
                profile_data.get("marital_status", ""),
                profile_data.get("education", ""),
                profile_data.get("job", ""),
                profile_data.get("location", ""),
                interests[0].get("interest", ""),
                interests[0].get("percentage", 0),
                interests[1].get("interest", ""),
                interests[1].get("percentage", 0),
                interests[2].get("interest", ""),
                interests[2].get("percentage", 0),
                profile_data.get("personality_summary", ""),
                json.dumps(profile_data.get("key_activities", [])),
                profile_data.get("total_posts", 0),
                json.dumps(profile_data.get("top_habits", [])),
                profile_data.get("top_hobby", ""),
                profile_data.get("travel_indicators", ""),
                json.dumps(profile_data.get("life_indicators", [])),
                json.dumps(profile_data.get("spending_indicators", []))
            )

            self.cursor.execute(sql, values)
            self.connection.commit()
            print("✅ Profile inserted successfully")
            return True

        except sqlite3.Error as e:
            self.connection.rollback()
            print(f"❌ Database error: {e}")
            return False
        except Exception as e:
            self.connection.rollback()
            print(f"❌ Unexpected error: {e}")
            return False
        finally:
            self.close()

    def close(self):
        """
        Close the database connection and cursor.
        """
        try:
            if hasattr(self, 'cursor'):
                self.cursor.close()
            if hasattr(self, 'connection'):
                self.connection.close()
            print("🔌 Database connection closed")
        except Exception as e:
            print(f"❌ Error closing connection: {e}")


In [6]:
db = Database("test_profiles.db")


🔌 Database connection successful


In [3]:

sample_profile = {
    "first_name": "John",
    "last_name": "Doe",
    "age": 28,
    "gender": "Male",
    "marital_status": "Single",
    "education": "Bachelor's in Computer Science",
    "job": "Software Engineer",
    "location": "Cairo, Egypt",
    "top_interests": [
        {"interest": "Technology", "percentage": 70},
        {"interest": "Traveling", "percentage": 20},
        {"interest": "Reading", "percentage": 10}
    ],
    "personality_summary": "Enthusiastic and proactive learner.",
    "key_activities": ["Coding", "Blogging", "Hiking"],
    "total_posts": 45,
    "top_habits": ["Morning run", "Reading news"],
    "top_hobby": "Photography",
    "travel_indicators": "Frequent traveler",
    "life_indicators": ["Outgoing", "Family-oriented"],
    "spending_indicators": ["Budget-conscious"]
}

In [4]:
inserted = db.insert_user_profile(sample_profile)
print(f"Insert successful: {inserted}")



✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True


In [4]:
sample_profiles = [
    {
        "first_name": "Aya",
        "last_name": "Shaker",
        "age": 22,
        "gender": "Female",
        "marital_status": "Single",
        "education": "High School",
        "job": "Intern Graphic Designer",
        "location": "Cairo, Egypt",
        "top_interests": [
            {"interest": "Design", "percentage": 60},
            {"interest": "Technology", "percentage": 25},
            {"interest": "Art", "percentage": 15}
        ],
        "personality_summary": "Creative and eager to learn design skills",
        "key_activities": ["Adobe Illustrator", "Behance portfolio", "UI/UX research"],
        "total_posts": 10,
        "top_habits": ["Design practice", "Skillshare courses"],
        "top_hobby": "Historical inspiration tours",
        "travel_indicators": "Local cultural explorer",
        "life_indicators": ["Career-focused", "Creative"],
        "spending_indicators": ["Education investments"]
    },
    {
        "first_name": "Nadia",
        "last_name": "Fawzy",
        "age": 34,
        "gender": "Female",
        "marital_status": "Divorced",
        "education": "BEd Arabic Literature",
        "job": "High School Teacher",
        "location": "Asyut, Egypt",
        "top_interests": [
            {"interest": "Education", "percentage": 65},
            {"interest": "Literature", "percentage": 25},
            {"interest": "Local Cuisine", "percentage": 10}
        ],
        "personality_summary": "Dedicated educator passionate about Arabic literature",
        "key_activities": ["Lesson planning", "Student development", "Cultural trips"],
        "total_posts": 8,
        "top_habits": ["Grading papers", "Arabic music"],
        "top_hobby": "Historical site visits",
        "travel_indicators": "Educational traveler",
        "life_indicators": ["Intellectual", "Student-focused"],
        "spending_indicators": ["Educational resources"]
    },
    {
        "first_name": "Hussein",
        "last_name": "Adel",
        "age": 20,
        "gender": "Male",
        "marital_status": "Single",
        "education": "Undergraduate BSc Computer Engineering",
        "job": "Student",
        "location": "Zagazig, Egypt",
        "top_interests": [
            {"interest": "Technology", "percentage": 70},
            {"interest": "Programming", "percentage": 20},
            {"interest": "Gaming", "percentage": 10}
        ],
        "personality_summary": "Tech-savvy student passionate about computing",
        "key_activities": ["FPGA projects", "Competitive programming", "Tech exploration"],
        "total_posts": 8,
        "top_habits": ["Late-night coding", "CS:GO sessions"],
        "top_hobby": "Hardware tinkering",
        "travel_indicators": "Occasional traveler",
        "life_indicators": ["Tech-focused", "Social gamer"],
        "spending_indicators": ["Tech gadgets"]
    },
    {
        "first_name": "Ramy",
        "last_name": "Youssef",
        "age": 45,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Civil Engineering",
        "job": "Project Director",
        "location": "Sohag, Egypt",
        "top_interests": [
            {"interest": "Engineering", "percentage": 60},
            {"interest": "Construction", "percentage": 30},
            {"interest": "Sports", "percentage": 10}
        ],
        "personality_summary": "Detail-oriented leader in construction management",
        "key_activities": ["Site supervision", "BIM coordination", "Safety compliance"],
        "total_posts": 10,
        "top_habits": ["Project monitoring", "Sports following"],
        "top_hobby": "Engineering innovation",
        "travel_indicators": "Project-based traveler",
        "life_indicators": ["Professional", "Family-oriented"],
        "spending_indicators": ["Professional tools"]
    },
    {
        "first_name": "Nahla",
        "last_name": "Hassan",
        "age": 27,
        "gender": "Female",
        "marital_status": "Single",
        "education": "BSc Fashion Design",
        "job": "Retail Buyer",
        "location": "Mansoura, Egypt",
        "top_interests": [
            {"interest": "Fashion", "percentage": 60},
            {"interest": "Retail", "percentage": 25},
            {"interest": "Visual Design", "percentage": 15}
        ],
        "personality_summary": "Trend-spotter with keen eye for fashion and retail",
        "key_activities": ["Fabric sourcing", "Market analysis", "Merchandising"],
        "total_posts": 10,
        "top_habits": ["Supplier meetings", "Store visual updates"],
        "top_hobby": "Cafe design inspiration",
        "travel_indicators": "Frequent work traveler",
        "life_indicators": ["Style-conscious", "Business-focused"],
        "spending_indicators": ["Fashion investments"]
    },
    {
        "first_name": "Ahmed",
        "last_name": "El-Sayed",
        "age": 35,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Computer Science",
        "job": "Software Engineer",
        "location": "Cairo, Egypt",
        "top_interests": [
            {"interest": "Technology", "percentage": 70},
            {"interest": "Travel", "percentage": 20},
            {"interest": "Food", "percentage": 10}
        ],
        "personality_summary": "Tech professional with diverse interests",
        "key_activities": ["Web development", "Hiking", "Culinary exploration"],
        "total_posts": 10,
        "top_habits": ["Tech upgrades", "Weekend adventures"],
        "top_hobby": "Blogging about tech",
        "travel_indicators": "Regular traveler",
        "life_indicators": ["Outgoing", "Family-oriented"],
        "spending_indicators": ["Tech and experiences"]
    },
    {
        "first_name": "Fatima",
        "last_name": "Hassan",
        "age": 28,
        "gender": "Female",
        "marital_status": "Single",
        "education": "BA Literature",
        "job": "Teacher",
        "location": "Alexandria, Egypt",
        "top_interests": [
            {"interest": "Education", "percentage": 60},
            {"interest": "Literature", "percentage": 25},
            {"interest": "Music", "percentage": 15}
        ],
        "personality_summary": "Inspiring educator with cultural appreciation",
        "key_activities": ["Lesson planning", "Library visits", "Music appreciation"],
        "total_posts": 10,
        "top_habits": ["Grading papers", "Cultural exploration"],
        "top_hobby": "Mythology studies",
        "travel_indicators": "Cultural traveler",
        "life_indicators": ["Intellectual", "Community-focused"],
        "spending_indicators": ["Books and education"]
    },
    {
        "first_name": "Mohamed",
        "last_name": "Ali",
        "age": 42,
        "gender": "Male",
        "marital_status": "Divorced",
        "education": "MBA",
        "job": "Business Consultant",
        "location": "Mansoura, Egypt",
        "top_interests": [
            {"interest": "Finance", "percentage": 60},
            {"interest": "Sports", "percentage": 25},
            {"interest": "Automotive", "percentage": 15}
        ],
        "personality_summary": "Strategic thinker with business acumen",
        "key_activities": ["Market analysis", "Sports events", "Car testing"],
        "total_posts": 10,
        "top_habits": ["Business conferences", "Tennis"],
        "top_hobby": "Podcast creation",
        "travel_indicators": "Business traveler",
        "life_indicators": ["Analytical", "Social"],
        "spending_indicators": ["Business investments"]
    },
    {
        "first_name": "Nadia",
        "last_name": "Khalil",
        "age": 25,
        "gender": "Female",
        "marital_status": "Single",
        "education": "BSc Pharmacy",
        "job": "Pharmacist",
        "location": "Giza, Egypt",
        "top_interests": [
            {"interest": "Health", "percentage": 60},
            {"interest": "Beauty", "percentage": 25},
            {"interest": "Cooking", "percentage": 15}
        ],
        "personality_summary": "Health professional with holistic wellness approach",
        "key_activities": ["Patient care", "Skincare research", "Culinary experiments"],
        "total_posts": 10,
        "top_habits": ["Wellness workshops", "Cafe socializing"],
        "top_hobby": "Herbal medicine studies",
        "travel_indicators": "Resort traveler",
        "life_indicators": ["Health-conscious", "Social"],
        "spending_indicators": ["Beauty and wellness"]
    },
    {
        "first_name": "Khaled",
        "last_name": "Mostafa",
        "age": 30,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Engineering",
        "job": "Civil Engineer",
        "location": "Aswan, Egypt",
        "top_interests": [
            {"interest": "Engineering", "percentage": 60},
            {"interest": "Sports", "percentage": 25},
            {"interest": "History", "percentage": 15}
        ],
        "personality_summary": "Practical engineer with historical appreciation",
        "key_activities": ["Construction oversight", "Football", "Temple visits"],
        "total_posts": 10,
        "top_habits": ["Project management", "Nile walks"],
        "top_hobby": "Diving exploration",
        "travel_indicators": "Domestic explorer",
        "life_indicators": ["Adventurous", "Family-oriented"],
        "spending_indicators": ["Professional development"]
    },
    {
        "first_name": "Laila",
        "last_name": "Abdelrahman",
        "age": 38,
        "gender": "Female",
        "marital_status": "Widowed",
        "education": "BA Psychology",
        "job": "Therapist",
        "location": "Luxor, Egypt",
        "top_interests": [
            {"interest": "Mental Health", "percentage": 70},
            {"interest": "Wellness", "percentage": 20},
            {"interest": "Music", "percentage": 10}
        ],
        "personality_summary": "Compassionate therapist dedicated to healing",
        "key_activities": ["Therapy sessions", "Mindfulness practice", "Cultural events"],
        "total_posts": 10,
        "top_habits": ["Classical music", "Reading"],
        "top_hobby": "Emotional intelligence studies",
        "travel_indicators": "Cultural traveler",
        "life_indicators": ["Compassionate", "Community-oriented"],
        "spending_indicators": ["Self-improvement"]
    },
    {
        "first_name": "Omar",
        "last_name": "El-Masry",
        "age": 22,
        "gender": "Male",
        "marital_status": "Single",
        "education": "High School",
        "job": "Student",
        "location": "Sharm El-Sheikh, Egypt",
        "top_interests": [
            {"interest": "Sports", "percentage": 60},
            {"interest": "Music", "percentage": 25},
            {"interest": "Fashion", "percentage": 15}
        ],
        "personality_summary": "Active student enjoying coastal lifestyle",
        "key_activities": ["Surfing", "Music exploration", "Social events"],
        "total_posts": 10,
        "top_habits": ["Exam preparation", "Beach activities"],
        "top_hobby": "Content creation",
        "travel_indicators": "Local explorer",
        "life_indicators": ["Active", "Social"],
        "spending_indicators": ["Fashion and experiences"]
    },
    {
        "first_name": "Yasmine",
        "last_name": "Farouk",
        "age": 45,
        "gender": "Female",
        "marital_status": "Married",
        "education": "BSc Architecture",
        "job": "Architect",
        "location": "Hurghada, Egypt",
        "top_interests": [
            {"interest": "Architecture", "percentage": 60},
            {"interest": "History", "percentage": 25},
            {"interest": "Food", "percentage": 15}
        ],
        "personality_summary": "Creative architect inspired by historical design",
        "key_activities": ["Design projects", "Historical research", "Culinary exploration"],
        "total_posts": 10,
        "top_habits": ["Site visits", "Beach walks"],
        "top_hobby": "International cuisine",
        "travel_indicators": "Frequent traveler",
        "life_indicators": ["Creative", "Family-oriented"],
        "spending_indicators": ["Design resources"]
    },
    {
        "first_name": "Hassan",
        "last_name": "Ibrahim",
        "age": 50,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Agriculture",
        "job": "Farmer",
        "location": "Fayoum, Egypt",
        "top_interests": [
            {"interest": "Agriculture", "percentage": 70},
            {"interest": "Sustainability", "percentage": 20},
            {"interest": "Automotive", "percentage": 10}
        ],
        "personality_summary": "Traditional farmer embracing modern techniques",
        "key_activities": ["Crop management", "Equipment maintenance", "Local markets"],
        "total_posts": 10,
        "top_habits": ["Farm work", "Market research"],
        "top_hobby": "Countryside exploration",
        "travel_indicators": "Occasional traveler",
        "life_indicators": ["Hardworking", "Family-oriented"],
        "spending_indicators": ["Farm equipment"]
    },
    {
        "first_name": "Amina",
        "last_name": "El-Gendy",
        "age": 33,
        "gender": "Female",
        "marital_status": "Divorced",
        "education": "BA Fine Arts",
        "job": "Artist",
        "location": "Port Said, Egypt",
        "top_interests": [
            {"interest": "Art", "percentage": 70},
            {"interest": "Fashion", "percentage": 20},
            {"interest": "Music", "percentage": 10}
        ],
        "personality_summary": "Expressive artist with unique creative vision",
        "key_activities": ["Painting", "Fashion events", "Jazz appreciation"],
        "total_posts": 10,
        "top_habits": ["Studio work", "Coastal walks"],
        "top_hobby": "Art instruction",
        "travel_indicators": "Gallery traveler",
        "life_indicators": ["Creative", "Independent"],
        "spending_indicators": ["Art supplies"]
    },
    {
        "first_name": "Tarek",
        "last_name": "El-Sherif",
        "age": 27,
        "gender": "Male",
        "marital_status": "Single",
        "education": "BSc Medicine",
        "job": "Doctor",
        "location": "Suez, Egypt",
        "top_interests": [
            {"interest": "Health", "percentage": 60},
            {"interest": "Fitness", "percentage": 25},
            {"interest": "Travel", "percentage": 15}
        ],
        "personality_summary": "Dedicated medical professional balancing work and wellness",
        "key_activities": ["Patient care", "Running", "Medical conferences"],
        "total_posts": 10,
        "top_habits": ["Hospital shifts", "Fitness routine"],
        "top_hobby": "Mountain hiking",
        "travel_indicators": "Adventure traveler",
        "life_indicators": ["Health-focused", "Active"],
        "spending_indicators": ["Medical education"]
    },
    {
        "first_name": "Mona",
        "last_name": "El-Din",
        "age": 40,
        "gender": "Female",
        "marital_status": "Married",
        "education": "BA History",
        "job": "Museum Curator",
        "location": "Aswan, Egypt",
        "top_interests": [
            {"interest": "History", "percentage": 70},
            {"interest": "Culture", "percentage": 20},
            {"interest": "Cooking", "percentage": 10}
        ],
        "personality_summary": "Knowledge curator preserving cultural heritage",
        "key_activities": ["Exhibit design", "Historical research", "Culinary experiments"],
        "total_posts": 10,
        "top_habits": ["Museum work", "Nile walks"],
        "top_hobby": "Nubian culture studies",
        "travel_indicators": "History-focused traveler",
        "life_indicators": ["Intellectual", "Family-oriented"],
        "spending_indicators": ["Cultural resources"]
    },
    {
        "first_name": "Ali",
        "last_name": "El-Mahdy",
        "age": 55,
        "gender": "Male",
        "marital_status": "Widowed",
        "education": "BSc Law",
        "job": "Lawyer",
        "location": "Tanta, Egypt",
        "top_interests": [
            {"interest": "Law", "percentage": 60},
            {"interest": "Sports", "percentage": 25},
            {"interest": "Automotive", "percentage": 15}
        ],
        "personality_summary": "Experienced legal professional with sports enthusiasm",
        "key_activities": ["Court appearances", "Football matches", "Car maintenance"],
        "total_posts": 10,
        "top_habits": ["Legal research", "Golf"],
        "top_hobby": "Real estate investment",
        "travel_indicators": "Business traveler",
        "life_indicators": ["Professional", "Community involvement"],
        "spending_indicators": ["Real estate"]
    },
    {
        "first_name": "Sara",
        "last_name": "El-Naggar",
        "age": 24,
        "gender": "Female",
        "marital_status": "Single",
        "education": "BA Marketing",
        "job": "Marketing Specialist",
        "location": "Zagazig, Egypt",
        "top_interests": [
            {"interest": "Marketing", "percentage": 60},
            {"interest": "Fashion", "percentage": 25},
            {"interest": "Music", "percentage": 15}
        ],
        "personality_summary": "Creative marketer with trend awareness",
        "key_activities": ["Campaign management", "Fashion events", "Pop music"],
        "total_posts": 10,
        "top_habits": ["Social media monitoring", "Shopping"],
        "top_hobby": "Trend blogging",
        "travel_indicators": "Fashion event traveler",
        "life_indicators": ["Social", "Career-focused"],
        "spending_indicators": ["Fashion and lifestyle"]
    },
    {
        "first_name": "Khaled",
        "last_name": "El-Sayed",
        "age": 32,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Physics",
        "job": "Researcher",
        "location": "Beni Suef, Egypt",
        "top_interests": [
            {"interest": "Science", "percentage": 70},
            {"interest": "Fitness", "percentage": 20},
            {"interest": "Travel", "percentage": 10}
        ],
        "personality_summary": "Analytical researcher with health consciousness",
        "key_activities": ["Experiments", "Biking", "Conferences"],
        "total_posts": 10,
        "top_habits": ["Research work", "Nutrition focus"],
        "top_hobby": "Science education",
        "travel_indicators": "Conference traveler",
        "life_indicators": ["Intellectual", "Family-oriented"],
        "spending_indicators": ["Research materials"]
    },
    {
        "first_name": "Hany",
        "last_name": "Kamal",
        "age": 39,
        "gender": "Male",
        "marital_status": "Married",
        "education": "BSc Electrical Engineering",
        "job": "Maintenance Manager",
        "location": "Port Said, Egypt",
        "top_interests": [
            {"interest": "Technology", "percentage": 60},
            {"interest": "Engineering", "percentage": 25},
            {"interest": "Sports", "percentage": 15}
        ],
        "personality_summary": "Practical engineer focused on system optimization",
        "key_activities": ["System upgrades", "Maintenance planning", "Sports events"],
        "total_posts": 5,
        "top_habits": ["Equipment monitoring", "Sports following"],
        "top_hobby": "Hybrid vehicle technology",
        "travel_indicators": "Occasional traveler",
        "life_indicators": ["Technical", "Family-oriented"],
        "spending_indicators": ["Technical tools"]
    }
]

In [8]:
for sample_profile in sample_profiles : 
    db = Database("test_profiles.db")
    inserted = db.insert_user_profile(sample_profile)
    print(f"Insert successful: {inserted}")


🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True
🔌 Database connection successful
✅ Table verified/created succes

In [5]:
# Verify insertion by querying directly
conn = sqlite3.connect("test_profiles.db")
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute("SELECT * FROM user_profiles")
rows = cur.fetchall()
for row in rows:
    print(dict(row))
cur.close()
conn.close()

{'id': 1, 'first_name': 'John', 'last_name': 'Doe', 'age': 28, 'gender': 'Male', 'marital_status': 'Single', 'education': "Bachelor's in Computer Science", 'job': 'Software Engineer', 'location': 'Cairo, Egypt', 'first_interest': 'Technology', 'first_interest_percentage': 70, 'second_interest': 'Traveling', 'second_interest_percentage': 20, 'third_interest': 'Reading', 'third_interest_percentage': 10, 'personality_summary': 'Enthusiastic and proactive learner.', 'key_activities': '["Coding", "Blogging", "Hiking"]', 'total_posts': 45, 'top_habits': '["Morning run", "Reading news"]', 'top_hobby': 'Photography', 'travel_indicators': 'Frequent traveler', 'life_indicators': '["Outgoing", "Family-oriented"]', 'spending_indicators': '["Budget-conscious"]', 'created_at': '2025-07-09 21:52:41'}


In [9]:

import streamlit as st
import pandas as pd
import sqlite3

@st.cache_data
def load_user_profiles():
    conn = sqlite3.connect("test_profiles.db")
    df = pd.read_sql_query("SELECT * FROM user_profiles", conn)
    print(df.head())
    conn.close()
    return df

# Load profiles dataframe
user_df = load_user_profiles()


2025-07-11 14:16:14.828 No runtime found, using MemoryCacheStorageManager


   id first_name last_name  age  gender marital_status  \
0   1       John       Doe   28    Male         Single   
1   2        Aya    Shaker   22  Female         Single   
2   3        Aya    Shaker   22  Female         Single   
3   4      Nadia     Fawzy   34  Female       Divorced   
4   5    Hussein      Adel   20    Male         Single   

                                education                      job  \
0          Bachelor's in Computer Science        Software Engineer   
1                             High School  Intern Graphic Designer   
2                             High School  Intern Graphic Designer   
3                   BEd Arabic Literature      High School Teacher   
4  Undergraduate BSc Computer Engineering                  Student   

         location first_interest  ...  third_interest_percentage  \
0    Cairo, Egypt     Technology  ...                         10   
1    Cairo, Egypt         Design  ...                         15   
2    Cairo, Egypt         

In [10]:
user_df

,id,first_name,last_name,age,gender,marital_status,education,job,location,first_interest,...,third_interest_percentage,personality_summary,key_activities,total_posts,top_habits,top_hobby,travel_indicators,life_indicators,spending_indicators,created_at
0,1,John,Doe,28,Male,Single,Bachelor's in Computer Science,Software Engineer,"Cairo, Egypt",Technology,...,10,Enthusiastic and proactive learner.,"[""Coding"", ""Blogging"", ""Hiking""]",45,"[""Morning run"", ""Reading news""]",Photography,Frequent traveler,"[""Outgoing"", ""Family-oriented""]","[""Budget-conscious""]",2025-07-09 21:52:41
1,2,Aya,Shaker,22,Female,Single,High School,Intern Graphic Designer,"Cairo, Egypt",Design,...,15,Creative and eager to learn design skills,"[""Adobe Illustrator"", ""Behance portfolio"", ""UI...",10,"[""Design practice"", ""Skillshare courses""]",Historical inspiration tours,Local cultural explorer,"[""Career-focused"", ""Creative""]","[""Education investments""]",2025-07-11 11:13:58
2,3,Aya,Shaker,22,Female,Single,High School,Intern Graphic Designer,"Cairo, Egypt",Design,...,15,Creative and eager to learn design skills,"[""Adobe Illustrator"", ""Behance portfolio"", ""UI...",10,"[""Design practice"", ""Skillshare courses""]",Historical inspiration tours,Local cultural explorer,"[""Career-focused"", ""Creative""]","[""Education investments""]",2025-07-11 11:16:03
3,4,Nadia,Fawzy,34,Female,Divorced,BEd Arabic Literature,High School Teacher,"Asyut, Egypt",Education,...,10,Dedicated educator passionate about Arabic lit...,"[""Lesson planning"", ""Student development"", ""Cu...",8,"[""Grading papers"", ""Arabic music""]",Historical site visits,Educational traveler,"[""Intellectual"", ""Student-focused""]","[""Educational resources""]",2025-07-11 11:16:03
4,5,Hussein,Adel,20,Male,Single,Undergraduate BSc Computer Engineering,Student,"Zagazig, Egypt",Technology,...,10,Tech-savvy student passionate about computing,"[""FPGA projects"", ""Competitive programming"", ""...",8,"[""Late-night coding"", ""CS:GO sessions""]",Hardware tinkering,Occasional traveler,"[""Tech-focused"", ""Social gamer""]","[""Tech gadgets""]",2025-07-11 11:16:03
5,6,Ramy,Youssef,45,Male,Married,BSc Civil Engineering,Project Director,"Sohag, Egypt",Engineering,...,10,Detail-oriented leader in construction management,"[""Site supervision"", ""BIM coordination"", ""Safe...",10,"[""Project monitoring"", ""Sports following""]",Engineering innovation,Project-based traveler,"[""Professional"", ""Family-oriented""]","[""Professional tools""]",2025-07-11 11:16:03
6,7,Nahla,Hassan,27,Female,Single,BSc Fashion Design,Retail Buyer,"Mansoura, Egypt",Fashion,...,15,Trend-spotter with keen eye for fashion and re...,"[""Fabric sourcing"", ""Market analysis"", ""Mercha...",10,"[""Supplier meetings"", ""Store visual updates""]",Cafe design inspiration,Frequent work traveler,"[""Style-conscious"", ""Business-focused""]","[""Fashion investments""]",2025-07-11 11:16:03
7,8,Ahmed,El-Sayed,35,Male,Married,BSc Computer Science,Software Engineer,"Cairo, Egypt",Technology,...,10,Tech professional with diverse interests,"[""Web development"", ""Hiking"", ""Culinary explor...",10,"[""Tech upgrades"", ""Weekend adventures""]",Blogging about tech,Regular traveler,"[""Outgoing"", ""Family-oriented""]","[""Tech and experiences""]",2025-07-11 11:16:03
8,9,Fatima,Hassan,28,Female,Single,BA Literature,Teacher,"Alexandria, Egypt",Education,...,15,Inspiring educator with cultural appreciation,"[""Lesson planning"", ""Library visits"", ""Music a...",10,"[""Grading papers"", ""Cultural exploration""]",Mythology studies,Cultural traveler,"[""Intellectual"", ""Community-focused""]","[""Books and education""]",2025-07-11 11:16:03
9,10,Mohamed,Ali,42,Male,Divorced,MBA,Business Consultant,"Mansoura, Egypt",Finance,...,15,Strategic thinker with business acumen,"[""Market analysis"", ""Sports events"", ""Car test...",10,"[""Business conferences"", ""Tennis""]",Podcast creation,Business traveler,"[""Analytical"", ""Social""]",

# Mysql

In [ ]:
import pymysql
mydb = pymysql.connect(host="", user="", password="")

In [2]:
mycursor = mydb.cursor()

In [3]:
mycursor.execute("USE mydatabase")

0

In [6]:
mycursor.execute("""CREATE TABLE user_profiles (
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    age INT,
    gender VARCHAR(20),
    marital_status VARCHAR(50),
    education VARCHAR(100),
    job VARCHAR(100),
    location VARCHAR(100),
    
    -- Top interests as separate columns
    first_interest VARCHAR(50),
    first_interest_percentage INT,
    second_interest VARCHAR(50),
    second_interest_percentage INT,
    third_interest VARCHAR(50),
    third_interest_percentage INT,
    
    personality_summary TEXT,
    
    -- Activities (stored as JSON array)
    key_activities TEXT,
    
    total_posts INT,
    
    -- Habits (stored as JSON array)
    top_habits TEXT,
    
    top_hobby VARCHAR(100),
    travel_indicators VARCHAR(50),
    
    -- Life indicators (stored as JSON array)
    life_indicators TEXT,
    
    -- Spending indicators (stored as JSON array)
    spending_indicators TEXT
)""")

0

In [ ]:
from typing import Dict, Any
import pymysql
import json

class Database:
    def __init__(self, host: str, user: str, password: str, db: str):
        """Initialize the database connection"""
        self.connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            db=db,
            cursorclass=pymysql.cursors.DictCursor
        )
        self.cursor = self.connection.cursor()

    def insert_user_profile(self, profile_data: Dict[str, Any]) -> bool:
        """Insert a user profile into the database"""
        try:
            # Convert lists to JSON strings
            key_activities = json.dumps(profile_data.get("key_activities", []))
            top_habits = json.dumps(profile_data.get("top_habits", []))
            life_indicators = json.dumps(profile_data.get("life_indicators", []))
            spending_indicators = json.dumps(profile_data.get("spending_indicators", []))
            
            # Get interests from the top_interests list
            interests = profile_data.get("top_interests", [{} for _ in range(3)])
            
            sql = """
            INSERT INTO user_profiles (
                first_name, last_name, age, gender, marital_status, education, job, location,
                first_interest, first_interest_percentage,
                second_interest, second_interest_percentage,
                third_interest, third_interest_percentage,
                personality_summary, key_activities, total_posts, top_habits,
                top_hobby, travel_indicators, life_indicators, spending_indicators
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

            values = (
                profile_data.get("first_name"),
                profile_data.get("last_name"),
                profile_data.get("age"),
                profile_data.get("gender"),
                profile_data.get("marital_status"),
                profile_data.get("education"),
                profile_data.get("job"),
                profile_data.get("location"),
                interests[0].get("interest"),
                interests[0].get("percentage"),
                interests[1].get("interest"),
                interests[1].get("percentage"),
                interests[2].get("interest"),
                interests[2].get("percentage"),
                profile_data.get("personality_summary"),
                key_activities,
                profile_data.get("total_posts"),
                top_habits,
                profile_data.get("top_hobby"),
                profile_data.get("travel_indicators"),
                life_indicators,
                spending_indicators
            )

            self.cursor.execute(sql, values)
            self.connection.commit()
            self.logger.info(f"Successfully inserted profile for {profile_data.get('first_name')} {profile_data.get('last_name')}")
            return True

        except pymysql.MySQLError as e:
            self.logger.error(f"Error inserting user profile: {e}")
            return False
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
            return False
